In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

#Read the CSV file and check the content.
data=pd.read_csv("/kaggle/input/covid19-osaka-city-in-japan-data/covid19_osaka_daily_data.csv")
df=data.query('prefecture=="Osaka"')

day,death,positive=pd.to_datetime(df["date"]),df["fatal_daily"],df["positive_daily"]
positivenan=positive.to_numpy()


#normalize the positive numbers by using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
positive = positivenan.reshape(-1, 1)
scaler = scaler.fit(positive)
min_max_non = scaler.transform(positive)


In [ ]:
#make sequential data from 1 to 479
list=[]
[list.append([[i]]) for i in range(1,480)]

#Define LSTM model
seq_number=np.array(list)
length_of_sequence = seq_number.shape[1] 
in_out_neurons = 1
n_hidden = 300
model = Sequential()

model.add(LSTM(n_hidden, batch_input_shape=(None, length_of_sequence, in_out_neurons), return_sequences=False))
model.add(Dense(in_out_neurons))
model.add(Activation("linear"))
model.add(Activation("linear"))
optimizer="rmsprop"
model.compile(loss="mean_squared_error", optimizer=optimizer)
#convert seq_number into numpy array, and name it ser_number2
ser_number2=seq_number.flatten()

#start fitting (learning,training)
model.fit(seq_number,min_max_non,epochs=5000)



In [ ]:
#test the model
predicted = model.predict(seq_number)
predicted_for_matplot=predicted.flatten()

#make sequential data from 481 to 564 for future prediction
list_f=[]
[list_f.append([[i]]) for i in range(481,565)]
seq_number_f=np.array(list_f)
ser_number_f2=seq_number_f.flatten()

#predict the positive number of COVID-19 on 7/31/2021 in Osaka,Japan
predicted_f = model.predict(seq_number_f)

#make date data from 5/9/2021 to 7/31/2021
from datetime import datetime, timedelta
date_list_f = [datetime(2021, 5, 9) + timedelta(days=i) for i in range(84)]

#make predicted data into numpy array
future=predicted_f.flatten()
#show predicted values
show_prediction=scaler.inverse_transform(predicted_f)
show_prediction

In [ ]:
#Let's plot the prediction results on matplot.pyplot
comp_x = [datetime(2020, 1, 16) + timedelta(days=i) for i in range(563)]
comp_y=np.concatenate((predicted_for_matplot,future))
plt.figure()
plt.plot(day,min_max_non, color="b", label="Real Data")
plt.bar(comp_x,comp_y,color="r", label="Deep learning model's prediction")
plt.legend()
plt.show()
#Print the prediction result.
print("COVID-19 positive on 7/31/2021 in Osaka,Japan :",int(show_prediction[-1])," cases")